In [4]:
import os
import mido
from mido import MidiFile
# from rich import print

In [5]:
def stretch_midi_file(file: MidiFile, new_duration_seconds):
    print(f"rescaling file from {file.length:.02f}s to {new_duration_seconds:.02f}s ({new_duration_seconds / file.length:.03f})")
    # Calculate stretch factor based on the original duration
    stretch_factor = new_duration_seconds / file.length
    
    # Scale the time attribute of each message by the stretch factor
    for track in file.tracks:
        for msg in track:
            msg.time = int(msg.time * stretch_factor)

    return file

In [6]:
data_dir = os.path.join("..", "inputs", "all-time")
target_bpm = 100

file =  os.listdir(data_dir)[0]
midi_file_path = os.path.join(data_dir, file)
midi_file = mido.MidiFile(midi_file_path)
bpm = float(os.path.basename(midi_file_path).split('-')[1])

print(f"found bpm {bpm} for file {os.path.basename(midi_file_path)}")
new_message = mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(target_bpm), time=0)
tempo_added = False
midi_file_cp = midi_file    

with mido.open_output('Disklavier') as output: # type: ignore
    for msg in midi_file.play(meta_messages=False):
        print(msg)
        output.send(msg)
    
    for track in midi_file_cp.tracks:
        # Remove existing set_tempo messages
        for msg in track:
            if msg.type == 'set_tempo':
                print(f"removing set tempo message {msg}")
                track.remove(msg)
        
        # Add new set_tempo message to the first track
        if not tempo_added:
            print(f"adding message {new_message}")
            track.insert(0, new_message)
            tempo_added = True

    new_time = midi_file.length * bpm / target_bpm
    midi_file_cp = stretch_midi_file(midi_file_cp, new_time)

    midi_file_cp.save("tmp.mid")
    midi_file_cp = MidiFile("tmp.mid")
    for msg in midi_file_cp.play(meta_messages=False):
        print(msg)
        output.send(msg)


found bpm 64.0 for file 20240124-064-02_0435-0442.mid
program_change channel=0 program=0 time=0
note_on channel=0 note=45 velocity=18 time=0.021306795454545457
note_on channel=0 note=61 velocity=60 time=0
note_on channel=0 note=54 velocity=24 time=0.4772722181818182
note_on channel=0 note=54 velocity=0 time=0.12784077272727273
note_on channel=0 note=61 velocity=0 time=0.6136357090909091
note_on channel=0 note=63 velocity=52 time=0
note_on channel=0 note=54 velocity=17 time=0.18749980000000002
note_on channel=0 note=61 velocity=49 time=0.008522718181818182
note_on channel=0 note=63 velocity=0 time=0.03835223181818182
note_on channel=0 note=61 velocity=0 time=0.16619300454545455
note_on channel=0 note=59 velocity=42 time=0.03835223181818182
note_on channel=0 note=45 velocity=0 time=0.06818174545454546
note_on channel=0 note=54 velocity=0 time=0.11931805454545455
note_on channel=0 note=47 velocity=34 time=0.008522718181818182
note_on channel=0 note=54 velocity=26 time=0.4900562954545455
n